# Unsupervised method to cluster collagen peptide masses for a range of unknown vertebrate species

First pre-processing step reads all CSV files available in a target root from 'monopeaklists' directories only and yields a binary matrix corresponding to protein peptide masses from a set range with a set bin size that can be changed to better suit the data.

In [1]:
#Import

import csv
import os
import glob
import fnmatch, re
import pandas as pd
import numpy as np

In [ ]:
#Preprocessing

R1 = 500
R2 = 4000
final = pd.DataFrame()
binsize = 1



regex = fnmatch.translate('*data*.csv')
reobj = re.compile(regex)

#os.walk -> collect and read all csv files required for clustering 
exclude = ['peaklists', 'relmonopeaklists'] #read only "monopeaklists"
for root, dirs, files in os.walk("."):
    dirs[:] = [d for d in dirs if d not in exclude]
    #print(dirs)
    for file in files:
        filename = os.path.join(root, file)
        print(filename)
        if file.endswith(".csv") and os.stat(filename).st_size > 20 and not reobj.match(file):
            #read all available CSV files (add exclusion of files < 20 bytes/ preprocessed data files)
            #filename = os.path.join(root, file)
            df1 = pd.read_csv(filename)
            print(filename)
            print(df1)
            i = df1['mass'].tolist()
            #print(df1.iloc[0])
            #print(i)
            mass = 0
            rangemass = []
            
            for item in i:
                if R1<float(float(item))<R2: #create labels with mass ranges
                    mass += float(item)
                    rangemass.append(float(item))
                    
            RL=R1
            num=int((R2-R1)/binsize)
            count = 0
            thisrange=['filename']
            numberofcount=[filename]
            
            for i in range (1,num+1): #count peaks per each bin
                RL=R1+1*(i-binsize)
                RH=RL+binsize
                wantrange=(RL,RH)
                thisrange.append(wantrange)
                for x in rangemass:
                    if RL<x<RH:
                        count=count+1
                numberofcount.append(count)
                count=0
            df2 = pd.DataFrame([numberofcount], columns=thisrange)
            print(df2)
            
            final = final.append(df2, ignore_index=True) #update dataframe -> final CSV file (binary matrix)
            final.to_csv("dataexample.csv", sep=',')


In [5]:
#data import/show

data = pd.read_csv('datalabel1.csv', index_col = [0])
data.head()



,filename,"(500, 501)","(501, 502)","(502, 503)","(503, 504)","(504, 505)","(505, 506)","(506, 507)","(507, 508)","(508, 509)",...,"(3990, 3991)","(3991, 3992)","(3992, 3993)","(3993, 3994)","(3994, 3995)","(3995, 3996)","(3996, 3997)","(3997, 3998)","(3998, 3999)","(3999, 4000)"
0,./20131112_P136sols/monopeaklists/20131112_P13...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,./20131112_P136sols/monopeaklists/20131112_P13...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,./20131112_P136sols/monopeaklists/20131112_P13...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,./20131112_P136sols/monopeaklists/20131112_P13...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,./20131112_P136sols/monopeaklists/20131112_P13...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
reduced1 = data.loc[:, (data != 0).any(axis=0)]
filelabels = reduced1.iloc[:,0]
reduced = reduced1.iloc[:,1:]
#print(reduced)
#reduced.shape